In [1]:
# Imports
import numpy as np
import torch

from phimal_utilities.data import Dataset
from phimal_utilities.data.burgers import BurgersDelta
from phimal_utilities.analysis import load_tensorboard

from DeePyMoD_SBL.deepymod_torch.library_functions import library_1D_in
from DeePyMoD_SBL.deepymod_torch.DeepMod import DeepModDynamic
from DeePyMoD_SBL.deepymod_torch.training import train_dynamic
from DeePyMoD_SBL.deepymod_torch.estimators import Threshold, Clustering, ClusteringL1, PDEFIND
from pysindy.optimizers import STLSQ

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [2]:
v = 0.1
A = 1.0

# Making grid
x = np.linspace(-3, 4, 100)
t = np.linspace(0.5, 5.0, 50)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')

dataset = Dataset(BurgersDelta, v=v, A=A)
X_train, y_train, rand_idx = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=1000, noise=0.2, random=True, return_idx=True)

In [3]:
theta = dataset.library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1))
dt = dataset.time_deriv(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1))

In [4]:
theta_normed = theta / np.linalg.norm(theta, axis=0, keepdims=True)

# Building train STLSQ method

In [5]:
from pysindy.optimizers import STLSQ
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [35]:
def TrainSTRidge(R, Ut, lam, d_tol, maxit = 25, STR_iters = 10, l0_penalty = None, normalize = 2, split = 0.8, print_best_tol = False):
        """
        This function trains a predictor using STRidge.

        It runs over different values of tolerance and trains predictors on a training set, then evaluates them 
        using a loss function on a holdout set.

        Please note published article has typo.  Loss function used here for model selection evaluates fidelity using 2-norm,
        not squared 2-norm.
        """

        # Split data into 80% training and 20% test, then search for the best tolderance.
        np.random.seed(0) # for consistancy
        n,_ = R.shape
        train = np.random.choice(n, int(n*split), replace = False)
        test = [i for i in np.arange(n) if i not in train]
        TrainR = R[train,:]
        TestR = R[test,:]
        TrainY = Ut[train,:]
        TestY = Ut[test,:]
        D = TrainR.shape[1]       

        # Set up the initial tolerance and l0 penalty
        d_tol = float(d_tol)
        tol = d_tol
        if l0_penalty == None: l0_penalty = 0.001*np.linalg.cond(R)

        # Get the standard least squares estimator
        w = np.zeros((D,1))
        w_best = np.linalg.lstsq(TrainR, TrainY)[0]
        err_best = np.linalg.norm(TestY - TestR.dot(w_best), 2) + l0_penalty*np.count_nonzero(w_best)
        tol_best = 0

        # Now increase tolerance until test performance decreases
        for iter in range(maxit):

            # Get a set of coefficients and error
            opt = STLSQ(threshold=tol, alpha=lam, fit_intercept=False)
            w = opt.fit(R, Ut).coef_.T
            err = np.linalg.norm(TestY - TestR.dot(w), 2) + l0_penalty*np.count_nonzero(w)

            # Has the accuracy improved?
            if err <= err_best:
                err_best = err
                w_best = w
                tol_best = tol
                tol = tol + d_tol

            else:
                tol = max([0,tol - 2*d_tol])
                d_tol  = 2*d_tol / (maxit - iter)
                tol = tol + d_tol
        return w_best

In [36]:
def TrainSTLSQ(X, y, alpha=1e-5, delta_threshold=1.0, max_iterations=25, test_size=0.2, random_state=0):
        '''Train STLSQ. Assumes data already normalized'''
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        
        # Set up the initial tolerance l0 penalty and estimates
        l0 = 1e-3 * np.linalg.cond(X)
        delta_t = delta_threshold # for interal use, can be updated
        optimizer = STLSQ(threshold=0, alpha=0.0, fit_intercept=False) # Now similar to LSTSQ
        
        # Initial estimate
        y_predict = optimizer.fit(X_train, y_train).predict(X_test)
        min_loss = np.linalg.norm(y_predict - y_test, 2) + l0 * np.count_nonzero(optimizer.coef_)
        
        # Now set parameters of optimizer and start thresholding
        optimizer = optimizer.set_params(threshold=delta_threshold, alpha=alpha)
        for iteration in np.arange(max_iterations):
            y_predict = optimizer.fit(X_train, y_train).predict(X_test)
            loss = np.linalg.norm(y_predict - y_test, 2) + l0 * np.count_nonzero(optimizer.coef_)
            
            if loss <= min_loss:
                min_loss = loss
                optimizer.set_params(threshold=optimizer.threshold + delta_t)
            else: # if loss increases, we need to a) lower the current threshold and/or decrease step size
                new_lower_tol = np.max([0, optimizer.threshold - 2 * delta_t])
                delta_t = 2 * delta_t / (max_iterations - iteration)
                optimizer.set_params(threshold=new_lower_tol + delta_t)

        return optimizer

In [46]:
noise = 0.5 * np.std(dt) * np.random.normal(size=dt.shape)

In [52]:
TrainSTLSQ(theta_normed, dt + noise).coef_.T

array([[ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [-9.48923445],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]])

In [53]:
TrainSTRidge(theta_normed, dt + noise, lam=1e-5, d_tol=1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


array([[  0.        ],
       [  0.        ],
       [  7.36456104],
       [  0.        ],
       [-10.59050087],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ]])